In [ ]:
!pip install backtrader

In [ ]:
!pip install alpaca_trade_api

In [ ]:
!pip install matplotlib==3.2.2

In [ ]:
!pip install plotly

In [ ]:
%%capture
!pip install backtrader
!pip install alpaca_trade_api
!pip install matplotlib==3.2.2
!pip install plotly

In [ ]:
import config
from alpaca_trade_api.rest import REST, TimeFrame
from alpaca_trade_api.stream import Stream
import plotly.graph_objects as go
import plotly.express as px
import alpaca_trade_api as trade_api 
import matplotlib.pyplot as plt
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# cradentials
API_KEY = config.API_KEY
SECRET_KEY = config.SECRET_KEY
BASE_URL = config.BASE_URL
data_url = 'wss://data.alpaca.markets'
ws_url = 'wss://data.alpaca.markets'

## Create the rest api object

In [ ]:

rest_api = REST(API_KEY, SECRET_KEY, BASE_URL)

## retrieve daily bar data for SPY in a dataframe 

In [ ]:
df_bars = rest_api.get_bars('NVDA', TimeFrame.Minute, '2023-01-01', '2023-01-07').df
df_bars.head(10)

In [ ]:
df_bars.to_csv("NVDA_minutes_2023_01_01.csv")

## Get account details

In [ ]:
rest_api.get_account()

## Closing all position at once

In [ ]:
rest_api.close_all_positions()

In [ ]:
# Reformat data (drop multiindex, rename columns, reset index)
df_bars.columns = df_bars.columns.to_flat_index()
df_bars.columns = [x[1] for x in df_bars.columns]
df_bars.reset_index(inplace=True)
print(df_bars.head())

# Plot stock price data
plot = df_bars.plot(x="timestamp", y="close", legend=False)
plot.set_xlabel("Date")
plot.set_ylabel("Apple Close Price ($)")
plt.show()


## submit buy order

In [ ]:
rest_api.submit_order(symbol='AAPL', qty=1, side='buy', type='market', time_in_force='day')

## submit sell order

In [ ]:

rest_api.submit_order(symbol='AAPL', qty=1, side='sell', type='market', time_in_force='day')

## get positions of portfolio

In [ ]:
portfolio_position = rest_api.get_position('AAPL')
print(portfolio_position)

## get a summery of all posessions

In [ ]:
portfolio = rest_api.list_positions()
# Print the quantity of shares for each position.
for position in portfolio:
    print("{} shares of {}".format(position.qty, position.symbol))

##  bar data candlestick plot

In [ ]:
# SPY bar data candlestick plot
candlestick_fig = go.Figure(data=[go.Candlestick(x=df_bars.index,
                                                 open=df_bars['open'],
                                                 high=df_bars['hhigh'],
                                                 low=df_bars['low'],
                                                 close=df_bars['close'])])

# calculating 5 day SMA using pandas rolling mean
sma = df_bars['close'].rolling(5).mean().dropna()

# creating a line plot for our sma
sma_fig = px.line(x=sma.index, y=sma)

# adding both plots onto one chart
fig = go.Figure(data=candlestick_fig.data + sma_fig.data)

# displaying our chart
fig.show()

In [ ]:
import backtrader as bt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 140 # chart resolution

def run_backtest(strategy, symbols, start, end, timeframe=TimeFrame.Day, cash=10000):
    '''params:
        strategy: the strategy you wish to backtest, an instance of backtrader.Strategy
        symbols: the symbol (str) or list of symbols List[str] you wish to backtest on
        start: start date of backtest in format 'YYYY-MM-DD'
        end: end date of backtest in format: 'YYYY-MM-DD'
        timeframe: the timeframe the strategy trades on (size of bars) -
                   1 min: TimeFrame.Minute, 1 day: TimeFrame.Day, 5 min: TimeFrame(5, TimeFrameUnit.Minute)
        cash: the starting cash of backtest
    '''

    # initialize backtrader broker
    cerebro = bt.Cerebro(stdstats=True)
    cerebro.broker.setcash(cash)

    # add strategy
    cerebro.addstrategy(strategy)

    # add analytics
    # cerebro.addobserver(bt.observers.Value)
    # cerebro.addobserver(bt.observers.BuySell)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
    
    # historical data request
    if type(symbols) == str:
        symbol = symbols
        alpaca_data = rest_api.get_bars(symbol, timeframe, start, end,  adjustment='all').df
        data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
        cerebro.adddata(data)
    elif type(symbols) == list or type(symbols) == set:
        for symbol in symbols:
            alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
            data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
            cerebro.adddata(data)

    # run
    initial_portfolio_value = cerebro.broker.getvalue()
    print(f'Starting Portfolio Value: {initial_portfolio_value}')
    results = cerebro.run()
    final_portfolio_value = cerebro.broker.getvalue()
    print(f'Final Portfolio Value: {final_portfolio_value} ---> Return: {(final_portfolio_value/initial_portfolio_value - 1)*100}%')

    strat = results[0]
    print('Sharpe Ratio:', strat.analyzers.mysharpe.get_analysis()['sharperatio'])
    cerebro.plot(iplot= False)

In [ ]:
class SmaCross(bt.Strategy):
  # list of parameters which are configurable for the strategy
    params = dict(
        pfast=13,  # period for the fast moving average
        pslow=25   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
  
    def next(self):
        if not self.position and self.crossover > 0:  # not in the market
            self.buy()
        elif self.position and self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position


run_backtest(SmaCross, 'AAPL', '2019-01-01', '2021-11-01', TimeFrame.Day, 10000)

In [ ]:
class AllWeatherStrategy(bt.Strategy):

    def __init__(self):
        # the last year we rebalanced (initialized to -1)
        self.year_last_rebalanced = -1 
        self.weights = { "VTI" : 0.30 , "TLT" : 0.40, "IEF": 0.15, "GLD" : 0.075, "DBC" : 0.075 }

    def next(self):
        # if we’ve already rebalanced this year
        if self.datetime.date().year == self.year_last_rebalanced:
            return
            
        # update year last balanced
        self.year_last_rebalanced = self.datetime.date().year
        
        # enumerate through each security
        for i,d in enumerate(self.datas):
            # rebalance portfolio with desired target percents
            symbol = d._name
            self.order_target_percent(d, target=self.weights[symbol])

run_backtest(AllWeatherStrategy, ["VTI", "TLT", "IEF", "GLD", "DBC"] , '2015-01-01', '2021-11-01', TimeFrame.Day, 10000)

In [ ]:
import random 

percent_allocations = {'VTI': 0.30, 'TLT': 0.40, 'IEF': 0.15, 'GLD': 0.075, 'DBC': 0.075}


# get total account value
account_equity = float(rest_api.get_account().equity)


# how many dollars we want to allocate to each symbol
dollar_value_allocations = {symbol: percent * account_equity for symbol, percent in percent_allocations.items()}

# liquidate all existing positions before rebalanc
rest_api.close_all_positions()


# Rebalance portfolio
for symbol, dollars_alloc in dollar_value_allocations.items():
     
    # market price of current ETF
    market_price = rest_api.get_latest_bar(symbol).close
    
    # how many shares we want
    target_holdings = dollars_alloc // market_price
     
    # how many shares we have to buy to match target
    order_quantity = target_holdings
     
    # submit market order for this ETF
    print(f"Submitting market order for {order_quantity} shares of {symbol}")
    rest_api.submit_order(symbol, order_quantity, 'buy', 'market', client_order_id=f'colab_{random.randrange(10000000)}')

# Stream bars 

In [ ]:
from statistics import mode
from alpaca_trade_api.stream import Stream
from datetime import datetime,timedelta
from anyio import current_time
import pandas as pd
import logging
import os
import time
import config
import nest_asyncio
nest_asyncio.apply()

API_KEY = config.API_KEY
SECRET_KEY = config.SECRET_KEY

In [ ]:
# dummy_path = dummy.__file__ 
async def trade_bars(bars):
    # handle all the trade/data manipulation
    temp_df = pd.DataFrame(
        columns=["Date", "Open", "High", "Low", "Close", "Volume", "Tic", "Vwap"]
    )
    
    present_time = datetime.utcfromtimestamp(bars.timestamp/10**9).strftime("%Y-%m-%d %H:%M:%S")
    temp_df["Date"] = [present_time]
    temp_df["Open"] = [bars.open]
    temp_df["High"] = [bars.high]
    temp_df["Low"] = [bars.low]
    temp_df["Close"] = [bars.close]
    temp_df["Volume"] = [bars.volume]
    temp_df["Tic"] = [bars.symbol]
#     temp_df["Exchange"] = [bars.exchange]
    temp_df["Vwap"] = [bars.vwap]

    temp_df.to_csv("bars.csv", mode="a", header=False)

    print(bars)
#     with open(dummy_path,"w") as fp:
#         fp.write(f"timestamp = '{datetime.now()}'")

def csv_handling(file_name: str, columns_list: list):
    if os.path.exists(file_name):
        try:
            trade_temp_df = pd.read_csv(file_name)
        except:
            print("The file doesn't exist, creating it")
            trade_temp_df = pd.DataFrame(columns=columns_list)
            trade_temp_df.to_csv(file_name)
        if trade_temp_df.empty:
            trade_temp_df = pd.DataFrame(columns=columns_list)
            trade_temp_df.to_csv(file_name)
        else:
            pass
    else:
        trade_temp_df = pd.DataFrame(columns=columns_list)
        trade_temp_df.to_csv(file_name)

def run_connection(stream):
    try:
        stream.run()
    except KeyboardInterrupt:
        print("Interrupted execution by the user")
        loop.run_until_complete(stream.stop_ws())
        exit(0)
    except Exception as e:
        print(f'Exception from websocket connection: {e}')
    finally:
        print('Trying to re-establish connection')
        time.sleep(3)
        run_connection(stream)

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    stream = Stream(
        API_KEY, SECRET_KEY, base_url="https://paper-api.alpaca.markets", raw_data=False,
        data_feed='iex',
#         crypto_exchanges = ['CBSE']
    )
    
    csv_handling(
        "bars.csv",
        columns_list=["time", "open", "high", "low", "close", "volume", "tic", "exchange","vwap"],
    )

    stream.subscribe_bars(trade_bars,'NVDA')
#     stream.subscribe_crypto_bars(trade_bars, "ETHUSD")
#     stream.subscribe_crypto_bars(trade_bars, "BTCUSD")
    run_connection(stream)
    print("Complete")